# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [7]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
import pandas as pd
load_dotenv(dotenv_path=os.path.expanduser(r"C:\Users\Zahinos\env"))

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [8]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [9]:
#Definition of the tables.
data = {
    'table': ['authors', 'books', 'publishers'],
    'definition': [
        'Contains information about book authors, including id, name, and nationality.',
        'Contains books details including title, author_id, publisher_id, and year of publication.',
        'Contains publisher details such as id, name, and country.'
    ]
}
df = pd.DataFrame(data)
print(df)

        table                                         definition
0     authors  Contains information about book authors, inclu...
1       books  Contains books details including title, author...
2  publishers  Contains publisher details such as id, name, a...


In [10]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [11]:
print(text_tables)

authors: Contains information about book authors, including id, name, and nationality.
books: Contains books details including title, author_id, publisher_id, and year of publication.
publishers: Contains publisher details such as id, name, and country.


In [13]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [17]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question= "Which authors have published books with publishers from Spain?")

In [18]:
print(return_OAI(pqt1))

{
    "tables": ["authors", "books", "publishers"]
}


In [19]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="List all book titles published after 2010.")

In [20]:
print(return_OAI(pqt3))

```json
{
    "tables": ["books"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?